# XGBoost Sample

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how to create and deploy IRIS classifier based on xgboost model into Merlin.

In [ ]:
!pip install --upgrade -r requirements.txt > /dev/null

In [ ]:
import merlin
import warnings
import os
import xgboost as xgb
from merlin.model import ModelType
from sklearn.datasets import load_iris
warnings.filterwarnings('ignore')

## 1. Initialize Merlin Resources

### 1.1 Set Merlin Server

In [ ]:
# Set Merlin Server
merlin.set_url("localhost:8080/api/merlin")

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP console at http://localhost:8080/projects/create.

In [ ]:
merlin.set_project("sample")

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in Merlin is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by Merlin are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
merlin.set_model("xgboost-sample", ModelType.XGBOOST)

## 2. Train Model And Deploy

### 2.1 Create Model Version and Upload Model

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_model(model_dir=model_dir)
v.finish()
```

In [ ]:
model_dir = "xgboost-model"
BST_FILE = "model.bst"

# Create new version of the model
with merlin.new_model_version() as v:
    iris = load_iris()
    y = iris['target']
    X = iris['data']
    dtrain = xgb.DMatrix(X, label=y)
    param = {'max_depth': 6,
                'eta': 0.1,
                'silent': 1,
                'nthread': 4,
                'num_class': 10,
                'objective': 'multi:softmax'
                }
    xgb_model = xgb.train(params=param, dtrain=dtrain)
    model_file = os.path.join((model_dir), BST_FILE)
    xgb_model.save_model(model_file)
    
    # Upload the serialized model to Merlin
    merlin.log_model(model_dir=model_dir)

### 2.2 Deploy Model

In [ ]:
endpoint = merlin.deploy(v)

### 2.3 Send Test Request

In [ ]:
%%bash -s "$endpoint.url"
curl -v -X POST $1 -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

### 2.4 Delete Deployment

In [ ]:
merlin.undeploy(v)
